In [1]:
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

/tmp/ipykernel_8317/1028975186.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [3]:
# root = 'D:/Dev/project/roofcounting/'
root = '/home/jthe/Dev/project/roofcounting/'

In [4]:
part_A_train = os.path.join(root, 'data/train_data', 'images')
part_A_test = os.path.join(root, 'data/test_data', 'images')
path_sets = [part_A_train]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, "*.png")):
        img_paths.append(img_path)

In [6]:
model = CSRNet()

/home/jthe/anaconda3/envs/roofcount/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jthe/anaconda3/envs/roofcount/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
model = model.cuda()

In [9]:
# checkpoint = torch.load('model_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

NameError: name 'checkpoint' is not defined

In [13]:
mae = 0
for i in range(len(img_paths)):
    img = 255.0 * F.to_tensor(Image.open(img_paths[i]).convert('RGB'))

    img[0,:,:]=img[0,:,:]-92.8207477031
    img[1,:,:]=img[1,:,:]-95.2757037428
    img[2,:,:]=img[2,:,:]-104.877445883
    img = img.cuda()
    #img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.png','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
    print(i,mae)
print (mae/len(img_paths))

0 35.982810974121094
1 84.88446044921875
2 101.28921127319336
3 146.06652069091797
4 175.07921600341797
5 338.0636520385742
6 376.10584259033203
7 452.0951919555664
8 827.5802993774414
9 918.5312118530273
10 1108.709815979004
11 1319.3517532348633
12 1704.9823989868164
13 1765.6164817810059
14 1864.8154411315918
15 2712.7965202331543
16 2818.105411529541
17 2836.8709602355957
18 2988.6063117980957
19 3034.017894744873
20 3064.470676422119
21 3256.475299835205
22 3469.54976272583
23 3584.1001625061035
24 3641.455638885498
25 3701.0410346984863
26 4615.097797393799
27 5208.267108917236
28 5256.054046630859
29 5266.795223236084
30 6096.771541595459
31 6150.8768882751465
32 6203.732109069824
33 6284.858848571777
34 6450.949150085449
35 6519.843399047852
36 6820.722671508789
37 7944.121109008789
38 8992.044326782227
39 9203.530349731445
40 9431.185653686523
41 9540.943969726562
42 9580.41901397705
43 9620.535701751709
44 9623.37195968628
45 9938.67826461792
46 10134.075969696045
47 10156.87